<a href="https://colab.research.google.com/github/Simo56a0/machineLEARNING/blob/master/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Show GPU name


True
Tesla T4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

folder_path = '/content/drive/MyDrive/DATASET_ug_sign_language'
files_in_folder = os.listdir(folder_path)
print(files_in_folder)


['sign_annotations.csv', 'videos', '.ipynb_checkpoints', 'checkpoint.txt', 'frames1', 'resized_frames', 'normalized_frames', 'keypoints', 'encoded_sign_labels.csv']


In [ ]:
dataset_path = '/content/drive/MyDrive/DATASET_ug_sign_language'


In [ ]:
import pandas as pd

# Load the CSV file
csv_path = "/content/drive/MyDrive/DATASET_ug_sign_language/sign_annotations.csv"  # Update path
df = pd.read_csv(csv_path)

# Display dataset structure
print(df.head())
print(f"Total Videos: {len(df)}")


   video_ID sign_word
0  0001.mp4     hello
1  0002.mp4       you
2  0003.mp4       how
3  0004.mp4      name
4  0005.mp4      what
Total Videos: 90


In [ ]:
import cv2
import os
import pandas as pd
import time

# Paths
video_folder = "/content/drive/MyDrive/DATASET_ug_sign_language/videos"
output_folder = "/content/drive/MyDrive/DATASET_ug_sign_language/frames1"
csv_path = "/content/drive/MyDrive/DATASET_ug_sign_language/sign_annotations.csv"

os.makedirs(output_folder, exist_ok=True)

# Load CSV
df = pd.read_csv(csv_path)

# Process videos in batches
BATCH_SIZE = 10
processed_count = 0

def extract_frames(video_path, output_path, video_ID):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"❌ ERROR: Cannot open video {video_path}")
        return  # Skip this video

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_filename = os.path.join(output_path, f"{video_ID}_frame_{frame_count}.jpg")
        cv2.imwrite(frame_filename, frame)
        frame_count += 1

    cap.release()
    print(f"✅ Extracted {frame_count} frames from {video_ID}")

# Process in batches
for index, row in df.iterrows():
    video_ID = row['video_ID']
    video_path = os.path.join(video_folder, f"{video_ID}")

    if os.path.exists(video_path):
        extract_frames(video_path, output_folder, video_ID)
        processed_count += 1
    else:
        print(f"❌ Video not found: {video_path}")

    if processed_count >= BATCH_SIZE:
        print("⏸️ Batch complete. Taking a short break...")
        time.sleep(10)  # Prevent Google Colab crash
        processed_count = 0  # Reset counter


✅ Extracted 61 frames from 0001.mp4
✅ Extracted 24 frames from 0002.mp4
✅ Extracted 51 frames from 0003.mp4
✅ Extracted 71 frames from 0004.mp4
✅ Extracted 55 frames from 0005.mp4
✅ Extracted 26 frames from 0006.mp4
✅ Extracted 75 frames from 0007.mp4
✅ Extracted 58 frames from 0008.mp4
✅ Extracted 93 frames from 0009.mp4
✅ Extracted 32 frames from 0010.mp4
⏸️ Batch complete. Taking a short break...
✅ Extracted 42 frames from 0011.mp4
✅ Extracted 47 frames from 0012.mp4
✅ Extracted 80 frames from 0013.mp4
✅ Extracted 56 frames from 0014.mp4
✅ Extracted 73 frames from 0015.mp4
✅ Extracted 86 frames from 0016.mp4
✅ Extracted 65 frames from 0017.mp4
✅ Extracted 28 frames from 0018.mp4
✅ Extracted 71 frames from 0019.mp4
✅ Extracted 63 frames from 0020.mp4
⏸️ Batch complete. Taking a short break...
✅ Extracted 63 frames from 0021.mp4
✅ Extracted 109 frames from 0022.mp4
✅ Extracted 46 frames from 0023.mp4
✅ Extracted 44 frames from 0024.mp4
✅ Extracted 74 frames from 0025.mp4
✅ Extracted 9

In [ ]:
import cv2
import os
import numpy as np

# Define new image size (adjust based on model input)
IMG_SIZE = (224, 224)

# Input & Output folders
frames_folder = "/content/drive/MyDrive/DATASET_ug_sign_language/frames1"
resized_folder = "/content/drive/MyDrive/DATASET_ug_sign_language/resized_frames/"

# Ensure output directory exists
os.makedirs(resized_folder, exist_ok=True)

# Process all frames
for frame_file in os.listdir(frames_folder):
    frame_path = os.path.join(frames_folder, frame_file)

    # Read the image
    img = cv2.imread(frame_path)
    if img is None:
        print(f"❌ Error reading {frame_file}")
        continue

    # Resize image
    img_resized = cv2.resize(img, IMG_SIZE)

    # Save resized image
    resized_path = os.path.join(resized_folder, frame_file)
    cv2.imwrite(resized_path, img_resized)

print("✅ All frames resized successfully!")


✅ All frames resized successfully!


In [ ]:
import cv2
import numpy as np
import os

# Normalization type (Choose one)
NORMALIZATION_TYPE = "0_to_1"  # Options: "0_to_1" or "-1_to_1"

# Paths
resized_folder = "/content/drive/MyDrive/DATASET_ug_sign_language/resized_frames"
normalized_folder = "/content/drive/MyDrive/DATASET_ug_sign_language/normalized_frames/"

# Ensure output directory exists
os.makedirs(normalized_folder, exist_ok=True)

# Process all resized images
for frame_file in os.listdir(resized_folder):
    frame_path = os.path.join(resized_folder, frame_file)

    # Read the image
    img = cv2.imread(frame_path)
    if img is None:
        print(f"❌ Error reading {frame_file}")
        continue

    # Convert to float32 and normalize
    img = img.astype(np.float32)

    if NORMALIZATION_TYPE == "0_to_1":
        img /= 255.0  # Scale to range [0,1]
    elif NORMALIZATION_TYPE == "-1_to_1":
        img = (img / 127.5) - 1  # Scale to range [-1,1]

    # Save normalized image (optional, since models can normalize during training)
    normalized_path = os.path.join(normalized_folder, frame_file)
    cv2.imwrite(normalized_path, (img * 255).astype(np.uint8))  # Convert back to uint8 for storage

print("✅ All frames normalized successfully!")


✅ All frames normalized successfully!


In [ ]:
import numpy as np
import cv2
import os

# Paths
normalized_folder = "/content/drive/MyDrive/DATASET_ug_sign_language/normalized_frames"
csv_path = "/content/drive/MyDrive/DATASET_ug_sign_language/sign_annotations.csv"

# Load CSV file
import pandas as pd
df = pd.read_csv(csv_path)

# Convert labels (sign words) to numerical values
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['sign_word'])

# Create X (frames) and Y (labels)
X = []
Y = []

for index, row in df.iterrows():
    video_id = row['video_ID']
    label = row['label']

    # Find all frames for this video
    frames = sorted([f for f in os.listdir(normalized_folder) if f.startswith(f"{video_id}_")])

    for frame_file in frames:
        frame_path = os.path.join(normalized_folder, frame_file)

        # Load image
        img = cv2.imread(frame_path)
        if img is None:
            continue

        # Convert to NumPy array
        X.append(img)
        Y.append(label)

# Convert to NumPy arrays
X = np.array(X)
Y = np.array(Y)

print(f"✅ Loaded {len(X)} frames into memory.")
print(f"X shape: {X.shape}, Y shape: {Y.shape}")


✅ Loaded 4247 frames into memory.
X shape: (4247, 224, 224, 3), Y shape: (4247,)


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the CSV file
csv_path = "/content/drive/MyDrive/DATASET_ug_sign_language/sign_annotations.csv"
df = pd.read_csv(csv_path)

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply encoding to the 'sign_word' column
df['encoded_label'] = label_encoder.fit_transform(df['sign_word'])

# Save mapping for later use (word → number)
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Save the encoded labels to a new CSV
df.to_csv("/content/drive/MyDrive/DATASET_ug_sign_language/encoded_sign_labels.csv", index=False)

# Show first few rows
print(df.head())
print("\nLabel Mapping:", label_mapping)


   video_ID sign_word  encoded_label
0  0001.mp4     hello             28
1  0002.mp4       you             84
2  0003.mp4       how             31
3  0004.mp4      name             46
4  0005.mp4      what             78

Label Mapping: {'about': 0, 'afternoon': 1, 'and': 2, 'any tips': 3, 'as follows': 4, 'bad': 5, 'book': 6, 'but': 7, 'can': 8, 'children': 9, 'come': 10, 'cook': 11, 'culture and tradition': 12, 'day': 13, 'different': 14, 'dinner': 15, 'directions': 16, 'enter': 17, 'evening': 18, 'family': 19, 'follows': 20, 'food': 21, 'forward': 22, 'game': 23, 'good': 24, 'goodbye': 25, 'guardians': 26, 'have': 27, 'hello': 28, 'help': 29, 'here': 30, 'how': 31, 'hungry': 32, 'interact': 33, 'is': 34, 'join us': 35, 'know': 36, 'learning': 37, 'looking': 38, 'lunch': 39, 'market': 40, 'me': 41, 'meet people': 42, 'members': 43, 'morning': 44, 'moving': 45, 'name': 46, 'next': 47, 'now': 48, 'numbers': 49, 'old': 50, 'open the door': 51, 'other': 52, 'our': 53, 'parents': 54, 'pl

In [ ]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Assuming X (frames) and Y (labels) are lists
X = np.array(X, dtype="float32")  # Convert frames to NumPy array
Y = np.array(Y, dtype="int")  # Convert labels to NumPy array

# Fix: Get number of unique classes correctly
NUM_CLASSES = len(np.unique(Y))  # ✅ Use np.unique instead of set(Y)

# One-hot encode labels
Y = to_categorical(Y, num_classes=NUM_CLASSES)

# Split dataset (80% training, 20% testing)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Save the test data
np.save("X_test.npy", X_test)
np.save("Y_test.npy", Y_test)

# Optional: Save training data
np.save("X_train.npy", X_train)
np.save("Y_train.npy", Y_train)

print(f"✅ Training Samples: {X_train.shape}, Testing Samples: {X_test.shape}")


✅ Training Samples: (3397, 224, 224, 3), Testing Samples: (850, 224, 224, 3)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout



In [ ]:
IMG_SIZE = 224  # Resize all frames to 64x64
NUM_CLASSES = 85  # Update this with your number of classes
DATA_PATH = "/content/drive/MyDrive/DATASET_ug_sign_language/normalized_frames"
CSV_PATH = "/content/drive/MyDrive/DATASET_ug_sign_language/encoded_sign_labels.csv"


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(32, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(512, activation='relu'),  # Reduced dense layer size
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
cnn_model = build_cnn_model(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=NUM_CLASSES)
cnn_model.summary()  # Check the updated model summary
cnn_model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=20, batch_size=32)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 222, 222, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 222, 222, 16)        │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 111, 111, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 109, 109, 32)        │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 109, 109, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 54, 54, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 93312)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │      47,776,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 85)                  │          43,605 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 47,825,141 (182.44 MB)

 Trainable params: 47,825,045 (182.44 MB)

 Non-trainable params: 96 (384.00 B)

Epoch 1/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 18s 111ms/step - accuracy: 0.2596 - loss: 20.4037 - val_accuracy: 0.1388 - val_loss: 17.5934
Epoch 2/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - accuracy: 0.7040 - loss: 1.7576 - val_accuracy: 0.8141 - val_loss: 1.0284
Epoch 3/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - accuracy: 0.7924 - loss: 1.4728 - val_accuracy: 0.9353 - val_loss: 0.3207
Epoch 4/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - accuracy: 0.8662 - loss: 0.9589 - val_accuracy: 0.9529 - val_loss: 0.3209
Epoch 5/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - accuracy: 0.8805 - loss: 0.8958 - val_accuracy: 0.8929 - val_loss: 1.0491
Epoch 6/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - accuracy: 0.8887 - loss: 0.9047 - val_accuracy: 0.9459 - val_loss: 0.5140
Epoch 7/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - accuracy: 0.9180 - loss: 0.4789 - val_accuracy: 0.9682 - val_loss: 0.3443
Epoch 8/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - accuracy: 0.9267 - loss: 0.6280 - val_

In [ ]:
cnn_model.save("sign_language_model.h5")
print("✅ Model saved successfully!")


✅ Model saved successfully!


In [ ]:
import tensorflow as tf

# Load the original model
model = tf.keras.models.load_model("sign_language_model.h5")

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Optional: Apply quantization (reduces size but may slightly impact accuracy)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

# Save the new model
with open("sign_language_model.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Model converted to TFLite successfully!")


Saved artifact at '/tmp/tmpsbhjep2d'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 85), dtype=tf.float32, name=None)
Captures:
  137120121426512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137120121422672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137120121426320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137120121422864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137120121426896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137120121425360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137120121427088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137119885351760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137119885355408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137119885350032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137119885353

In [ ]:
import os
os.listdir("/content/")


['.config',
 'X_train.npy',
 'cnn_model.joblib',
 'drive',
 'X_test.npy',
 'Y_test.npy',
 'sign_language_model.h5',
 'Y_train.npy',
 'sample_data']

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load your trained model
model = load_model("sign_language_model.h5")  # Replace with your actual model file

# Load and preprocess test data
X_test = np.load("X_test.npy")  # Replace with actual test dataset
y_test = np.load("Y_test.npy")  # True labels

# Make predictions
y_pred = model.predict(X_test)

# Convert y_test to class indices if it's one-hot encoded
if y_test.ndim == 3 and y_test.shape[2] > 1:  # Check if y_test is one-hot encoded
    y_test = np.argmax(y_test, axis=2)  # Convert to class indices
# If y_test has probabilities, you might want to threshold it to get class indices
#  y_test = (y_test > 0.5).astype(int)  # Example: Threshold at 0.5

# Convert Y_test and y_pred to appropriate format for accuracy_score
Y_test_labels = np.argmax(y_test, axis=1)  # Convert Y_test from one-hot to class indices
y_pred_labels = np.argmax(y_pred, axis=1)  # Convert predictions from one-hot to class indices

# Evaluate accuracy
accuracy = accuracy_score(Y_test_labels, y_pred_labels)  # Use class indices for accuracy
print(f"Accuracy: {accuracy:.4f}")

# Classification Report (use class indices for report)
print("\nClassification Report:")
print(classification_report(Y_test_labels, y_pred_labels, zero_division=1))  # Use class indices for report

# Confusion Matrix (use class indices for matrix)
print("\nConfusion Matrix:")
print(confusion_matrix(Y_test_labels, y_pred_labels))  # Use class indices for matrix

27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
Accuracy: 0.9600

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       1.00      1.00      1.00        14
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00        18
           4       1.00      1.00      1.00         9
           5       1.00      1.00      1.00         9
           6       1.00      1.00      1.00        10
           7       1.00      1.00      1.00         1
           8       1.00      1.00      1.00         8
           9       0.93      1.00      0.96        13
          10       1.00      0.89      0.94         9
          11       0.80      1.00      0.89         4
          12       0.88      1.00      0.93         7
          13       1.00      1.00      1.00        27
          14       0.86      1.00      0.92         6
          15       1.00      1.00      1.00         7


In [ ]:
!pip install huggingface_hub


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_file(
    path_or_fileobj="/content/sign_language_model.h5",  # Path to your model file
    path_in_repo="sign_language_model.h5",
    repo_id="simo2valid/cnnsignlanguagemodel",  # Change this to your Hugging Face username and repo name
    repo_type="model"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


sign_language_model.h5:   0%|          | 0.00/574M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/simo2valid/cnnsignlanguagemodel/commit/3d637e5d2a45056e80195954a82630e3c8c3f7fb', commit_message='Upload sign_language_model.h5 with huggingface_hub', commit_description='', oid='3d637e5d2a45056e80195954a82630e3c8c3f7fb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/simo2valid/cnnsignlanguagemodel', endpoint='https://huggingface.co', repo_type='model', repo_id='simo2valid/cnnsignlanguagemodel'), pr_revision=None, pr_num=None)

In [ ]:
pip install flask tensorflow


In [ ]:
from flask import Flask, request, jsonify
import tensorflow as tf
import numpy as np
import cv2

app = Flask(__name__)

# Load your trained model
model = tf.keras.models.load_model("sign_language_model.h5")

@app.route('/predict', methods=['POST'])
def predict():
    try:
        file = request.files['image']
        image = cv2.imdecode(np.frombuffer(file.read(), np.uint8), cv2.IMREAD_COLOR)
        image = cv2.resize(image, (64, 64)) / 255.0  # Resize and normalize
        image = np.expand_dims(image, axis=0)  # Add batch dimension

        # Make a prediction
        prediction = model.predict(image)
        predicted_class = np.argmax(prediction)

        return jsonify({"predicted_class": int(predicted_class)})

    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == '__main__':
    app.run(debug=True)


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
